In [1]:
!mkdir -p /content/drive
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
cd drive/Team\ Drives/Genre

/content/drive/Team Drives/Genre


In [4]:
!pip uninstall keras
!pip install keras==2.0.8
!pip install matplotlib2tikz

Uninstalling Keras-2.2.4:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/Keras-2.2.4.dist-info/*
    /usr/local/lib/python3.6/dist-packages/docs/*
    /usr/local/lib/python3.6/dist-packages/keras/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.6/dist-packages/docs/md_autogen.py
    /usr/local/lib/python3.6/dist-packages/docs/update_docs.py
Proceed (y/n)? y
  Successfully uninstalled Keras-2.2.4
    100% |████████████████████████████████| 276kB 8.9MB/s 
textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.0.8 which is incompatible.


In [5]:
cd recurrentshop/

/content/drive/Team Drives/Genre/recurrentshop


In [6]:
!python setup.py install

running install
running bdist_egg
running egg_info
writing recurrentshop.egg-info/PKG-INFO
writing dependency_links to recurrentshop.egg-info/dependency_links.txt
writing requirements to recurrentshop.egg-info/requires.txt
writing top-level names to recurrentshop.egg-info/top_level.txt
reading manifest file 'recurrentshop.egg-info/SOURCES.txt'
writing manifest file 'recurrentshop.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/recurrentshop
creating build/bdist.linux-x86_64/egg/recurrentshop/backend
copying build/lib/recurrentshop/backend/theano_backend.py -> build/bdist.linux-x86_64/egg/recurrentshop/backend
copying build/lib/recurrentshop/backend/tensorflow_backend.py -> build/bdist.linux-x86_64/egg/recurrentshop/backend
copying build/lib/recurrentshop/backend/__init__.py -> build/bdist.linux-x86_64/egg/recurrentshop/backend
copying build/lib/

In [7]:
cd ../

/content/drive/Team Drives/Genre


In [3]:
ls

'Copy of settings.py'          README.md
 credentials.json              recurrentshop/
 data/                         settings.py
 data_folder_setting.py        token.pickle
 Github.gdoc                   utils/
 instrument_classifier.ipynb   vae_definition.py
 LICENSE                       vae_evaluation.py
 models/                       vae_training.ipynb
 pickles/                      vae_training.py
 pitch_classifier.ipynb        velocity_classifier.ipynb
 __pycache__/


In [4]:
import keras
keras.__version__

Using TensorFlow backend.


'2.0.8'

In [0]:
# use big or small data
big = 1

fil = open('data_folder_setting.py', 'w')

if not big:
  fil.write('epochs = 1\n')
  fil.write('max_songs = 20\n')
  fil.write('source_folder = \'data/JvP_small\'\n')  # folder of data
  fil.write('classes = [\'Jazz\', \'Pop\']\n') # folder in source_folder

# big data
if big:
  fil.write('epochs = 100\n')
  fil.write('max_songs = 100\n')
  fil.write('source_folder = \'data/JvP\'\n')  # folder of data
  fil.write('classes = [\'Jazz\', \'Pop\']\n') # folder in source_folder
  #epochs = 2000
  #source_folder = 'data/JvP/'
  #classes = ['Jazz', 'Pop'] # folder in source_folder
fil.close()

In [0]:
from settings import *
from keras.models import Sequential
from keras import regularizers
from keras.layers import Input, RepeatVector
from keras.models import Model
from keras.layers.recurrent import LSTM, GRU
from keras.layers import TimeDistributed
from keras.layers import Dense, Activation
from keras.layers.embeddings import Embedding
from keras.optimizers import RMSprop, Adam
from keras.utils import to_categorical
from keras.layers.wrappers import Bidirectional
from random import shuffle
import progressbar
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import os
import numpy as np
import _pickle as pickle
import time
import utils.data_class as data_class
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
import pretty_midi as pm
import sys
from utils.import_midi import import_midi_from_folder
from matplotlib2tikz import save as tikz_save


In [7]:
# current data folder
print(epochs, source_folder, classes, max_songs)

100 data/JvP ['Jazz', 'Pop'] 100


In [0]:

model_path = 'models/pitchclustering/'
model_filetype = '.pickle'


verbose = False
show_plot = False
save_plot = True
lstm_size = 256
batch_size = 512
learning_rate = 0.00002 #1e-06
step_size = 1
save_step = 10
shuffle_train_set = True
bidirectional = False
embedding = False
optimizer = 'Adam'
activity_regularizer = None
reset_states = True
num_layers = 2
test_step = 1

In [9]:

print('loading data...')
# Get Train and test sets

folder = source_folder

V_train, V_test, D_train, D_test, T_train, T_test, I_train, I_test, Y_train, Y_test, X_train, X_test, C_train, C_test, train_paths, test_paths = import_midi_from_folder(folder)

train_set_size = len(X_train)
test_set_size = len(X_test)


print(len(train_paths))
print(len(test_paths))
print(C_test)

class_string = ''
for class_name in classes:
    class_string += class_name



fd = {'highcrop': high_crop, 'lowcrop':low_crop, 'lr': learning_rate, 'opt': optimizer,
'bi': bidirectional, 'lstm_size': lstm_size, 'trainsize': train_set_size, 
'testsize': test_set_size, 'input_length': input_length, 'reset_states': reset_states, 
'num_layers':num_layers, 'classes':class_string}
t = str(int(round(time.time())))
model_name = t+'-num_layers_%(num_layers)s_maxlen_%(input_length)s_lstmsize_%(lstm_size)s_trainsize_%(trainsize)s_testsize_%(testsize)s_classes_%(classes)s' % fd

model_path = model_path + model_name + '/'
if not os.path.exists(model_path):
    os.makedirs(model_path)


inputs = Input(shape=(None, input_dim))
lstm_outputs = inputs
for layer_no in range(num_layers-1):
    lstm_outputs = GRU(lstm_size, return_state=False, return_sequences=True)(lstm_outputs)
#last layer, that does not return sequences
lstm_outputs = GRU(lstm_size, return_state=False, return_sequences=False)(lstm_outputs)
dense = Dense(num_classes, activation='softmax')
outputs = dense(lstm_outputs)
model = Model(inputs, outputs)


loading data...
Importing Jazz song called moonlight_becomes_you_bl.mid
Importing Jazz song called smile_rl.mid
Importing Jazz song called that_sunday_that_summer-kar_db.mid
Importing Jazz song called a_string_of_pearls-rev_gw.mid
Importing Jazz song called harbor_lights_platters_rs.mid
Importing Jazz song called this_masquerade-GB-LR_oz.mid
Importing Jazz song called breakout_39gm-swing-out-sister_ps.mid
Importing Jazz song called ive_got_the_world_on_a_string-R2_gw.mid
Importing Jazz song called cant_we_be_friends-1929-kar_jpp.mid
Importing Jazz song called between_the_sheets_mellod.mid
Importing Jazz song called our_love_is_here_to_stay_ccm.mid
Importing Jazz song called saturday_night-1944-kar_jpp.mid
Importing Jazz song called it_takes_too_long-to_learn_to_live_alone-Eydie-Gorme-kar_rt.mid
Importing Jazz song called the_mta-the_kingston_trio-kar_rt.mid
Importing Jazz song called elmira_st_boogie_gr (1).mid
Importing Jazz song called siempre_hay_esperanza_jk.mid
Importing Jazz song

/usr/local/lib/python3.6/dist-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


Importing Jazz song called i_thought_about_you_dm.mid
Importing Jazz song called fly_by_night_rmb.mid
Importing Jazz song called always_rs-1.mid
Importing Jazz song called sing_sing_sing_prima_3x-gw.mid
Importing Jazz song called mc_modal_midi-Org_ee.mid
Importing Jazz song called black_orpheus_jh.mid
Importing Jazz song called romantic_dreams_pw.mid
Importing Jazz song called bossa_nova_usa_dwb.mid
Importing Jazz song called fools_rush_in_mw.mid
Importing Jazz song called maracangalha_sn.mid
Importing Jazz song called i_wish_you_love03_jlh.mid
Importing Jazz song called rum_n_coca_cola-andrews-sisters1945_ab.mid
Importing Jazz song called steppin_out-Joe-Jackson_eb.mid
Importing Jazz song called tico_tico_bh2.mid
Importing Jazz song called the_more_i_see_you-1945-v2-kar_jpp.mid
Importing Jazz song called im_old_fashioned-v1_dm.mid
Importing Jazz song called st_thomas-Sonny-Rollins_gt.mid
Importing Jazz song called count_on_him_jc.mid
Importing Jazz song called eye_of_the_beholder-chic

In [10]:

#compile autoencoder
if optimizer == 'RMS': optimizer = RMSprop(lr=learning_rate)
if optimizer == 'Adam': optimizer = Adam(lr=learning_rate)
loss = 'categorical_crossentropy'
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

print(model.summary())

# initialize loss arrays
total_test_loss_array = [] 
total_test_accuracy_array = []
total_train_loss_array = []
total_train_loss = 0
total_train_accuracy_array = []
total_train_accuracy = 0

# Test function
def test(testID):
    print('\nTesting:')
    total_test_loss = 0
    total_test_loss_length = 0
    total_test_loss_number = 0

    confusion_matrix = np.zeros((num_classes, num_classes))

    bar = progressbar.ProgressBar(max_value=test_set_size, redirect_stdout=False)
    for i, test_song in enumerate(X_test):

        X = X_test[i]
        num_samples = X.shape[0]
        c = C_test[i]
        Y = np.asarray([to_categorical(c, num_classes=num_classes)]*num_samples).squeeze()

        scores = model.evaluate(X,Y , batch_size=batch_size, verbose=verbose)
        if reset_states:
            model.reset_states()
        total_test_loss += scores[0]

        Y_predicted = model.predict(X, batch_size=batch_size, verbose=verbose)
        for y_val, y_predicted in zip(Y, Y_predicted):
            y_class_test = np.argmax(y_val)
            y_class_predicted = np.argmax(y_predicted)
            confusion_matrix[y_class_predicted, y_class_test] += 1
        bar.update(i+1)

    accuracy = np.sum(np.diagonal(confusion_matrix)) / np.sum(confusion_matrix)
    total_test_loss_array.append(total_test_loss/test_set_size)
    total_test_accuracy_array.append(accuracy)
    print('\nTotal test loss: ', total_test_loss/test_set_size)
    print('Total accuracy: ' + str(accuracy*100) + "%") 
    print('-'*50)
    plt.figure()
    plt.plot(total_test_loss_array, label='Total test loss')
    plt.plot(total_train_loss_array, label='Total train loss')
    plt.plot(total_test_accuracy_array, label='Total test accuracy')
    plt.plot(total_train_accuracy_array, label='Total train accuracy')
    plt.legend(loc='lower left', prop={'size': 8})
    if show_plot: plt.show()
    if save_plot: 
        plt.savefig(model_path+t+'pitch_classifier_train.png')
        tikz_save(model_path+t+'pitch_classifier_train.tex',encoding='utf-8',  show_info=False)

    pickle.dump(total_test_loss_array,open(model_path+'total_test_loss_array.pickle', 'wb'))
    pickle.dump(total_test_accuracy_array,open(model_path+'total_test_accuracy_array.pickle', 'wb'))
    pickle.dump(total_train_accuracy_array,open(model_path+'total_train_accuracy_array.pickle', 'wb'))
    pickle.dump(total_train_loss_array,open(model_path+'total_train_loss_array.pickle', 'wb'))

    if testID % save_step is 0:
        confusion_matrix = confusion_matrix/confusion_matrix.sum(axis=1, keepdims=True)
        plt.figure()
        plt.imshow(confusion_matrix, interpolation='nearest')
        plt.title('Total accuracy: ' + str(accuracy) + '%')
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        plt.xticks(np.arange(0,num_classes), classes)
        plt.yticks(np.arange(0,num_classes), classes)
        plt.colorbar()
        if show_plot: plt.show()
        if save_plot: 
            plt.savefig(model_path+t+'confusion_matrix' + str(testID) + '.png')
            tikz_save(model_path+t+'confusion_matrix' + str(testID) + '.tex', encoding='utf-8', show_info=False)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 61)          0         
_________________________________________________________________
gru_1 (GRU)                  (None, None, 256)         244224    
_________________________________________________________________
gru_2 (GRU)                  (None, 256)               393984    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 638,722
Trainable params: 638,722
Non-trainable params: 0
_________________________________________________________________
None


In [0]:

# Save Parameters to text file
with open(model_path + 'params.txt', "w", encoding='utf-8') as text_file:
    text_file.write("epochs: %s" % epochs + '\n')
    text_file.write("train_set_size: %s" % train_set_size + '\n')
    text_file.write("test_set_size: %s" % test_set_size + '\n')
    text_file.write("learning_rate: %s" % learning_rate + '\n')
    text_file.write("save_step: %s" % save_step + '\n')
    text_file.write("shuffle_train_set: %s" % shuffle_train_set + '\n')
    text_file.write("test_step: %s" % test_step + '\n')
    text_file.write("bidirectional: %s" % bidirectional + '\n')
    text_file.write("load_from_pickle_instead_of_midi: %s" % load_from_pickle_instead_of_midi + '\n')
    text_file.write("pickle_load_path: %s" % pickle_load_path + '\n')
    text_file.write("train_paths: %s" % train_paths + '\n')
    text_file.write("test_paths: %s" % test_paths + '\n')


# Train model
print('training model...')
for e in range(epochs):

    total_train_loss = 0
    total_train_accuracy = 0
    
    print('Epoch ', e, 'of ', epochs, 'Epochs\nTraining:')

    if shuffle_train_set:

        permutation = np.random.permutation(len(X_train))

        train_paths = [train_paths[i] for i in permutation]
        X_train = [X_train[i] for i in permutation]
        Y_train = [Y_train[i] for i in permutation]
        C_train = [C_train[i] for i in permutation]
        I_train = [I_train[i] for i in permutation]
        V_train = [V_train[i] for i in permutation]
        D_train = [D_train[i] for i in permutation]
        T_train = [T_train[i] for i in permutation]

    bar = progressbar.ProgressBar(max_value=train_set_size)
    
    # Train model with each song seperately
    for i, train_song in enumerate(X_train):

        X = X_train[i]

        num_samples = X.shape[0]

        if num_samples > 1:
            c = C_train[i]
            Y = np.asarray([to_categorical(c, num_classes=num_classes)]*num_samples).squeeze()


            hist = model.fit(X, Y,
                        epochs=1,
                        batch_size=batch_size,
                        shuffle=False,
                        verbose=verbose)

            if reset_states:
                model.reset_states()

            total_train_loss += np.mean(hist.history['loss'])
            total_train_accuracy += np.mean(hist.history['acc'])
        bar.update(i+1)
    if e%test_step is 0:
        total_train_loss = total_train_loss/train_set_size
        total_train_loss_array.append(total_train_loss)
        total_train_accuracy = total_train_accuracy/train_set_size
        total_train_accuracy_array.append(total_train_accuracy)
        test(e)
        

    if e%save_step is 0:
        print('saving model')
        model_save_path = model_path + 'model' + 'Epoch' + str(e) + model_filetype
        model.save(model_save_path)


training model...
Epoch  0 of  100 Epochs
Training:
Instructions for updating:
Use tf.cast instead.


100% (180 of 180) |######################| Elapsed Time: 0:03:55 ETA:  00:00:00


Testing:


100% (20 of 20) |########################| Elapsed Time: 0:00:11 ETA:  00:00:00


Total test loss:  0.6938866287469864
Total accuracy: 53.904282115869016%
--------------------------------------------------
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter
Non-automatic aspect ratio demanded, but neither height nor width of the plot are given. Discard aspect ratio.
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter
saving model
Epoch  1 of  100 Epochs
Training:


...

100% (180 of 180) |######################| Elapsed Time: 0:03:54 ETA:  00:00:00


Testing:


100% (20 of 20) |########################| Elapsed Time: 0:00:11 ETA:  00:00:00


Total test loss:  0.7312670841813087
Total accuracy: 49.52141057934509%
--------------------------------------------------
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter
Non-automatic aspect ratio demanded, but neither height nor width of the plot are given. Discard aspect ratio.
Horizontal alignment will be ignored as no 'x tick label text width' has been passed in the 'extra' parameter
Horizontal alignment will be ignored as no 'y tick label text width' has been passed in the 'extra' parameter
saving model
Epoch  21 of  100 Epochs
Training:


 10% (19 of 180) |##                     | Elapsed Time: 0:00:22 ETA:   0:02:24